In [1]:
import pandas as pd
import numpy as np
import torch
import os

from transformers import AutoTokenizer, AutoConfig, TrainingArguments, Trainer
from custom_dataset import LabelDataset, TestDataset
from datasets import load_metric, load_dataset
from classifier import RobertaForSequenceClassification
from sklearn.model_selection import StratifiedKFold
from utils import set_allseed
import warnings
import pickle
from augmentation import Augmentation

In [2]:
seed = 777
batch_size = 32
save_steps = 103
set_allseed(seed)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"
gpu = torch.device("cuda")
cpu = torch.device("cpu")
warnings.filterwarnings(action='ignore')
model_init = "klue/roberta-base"

In [3]:
def compute_metrics(pred):
    f1 = load_metric("f1")
    references = pred.label_ids
    predictions = pred.predictions.argmax(axis=1)
    metric = f1.compute(predictions=predictions, references=references, average="micro")
    return metric

In [4]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
train_df = train_df[["문장","label"]]

In [6]:
kfold_function = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
tokenizer = AutoTokenizer.from_pretrained(model_init)
config = AutoConfig.from_pretrained(model_init)
config.num_labels = len(train_df.label.value_counts())

In [7]:
aug = Augmentation(tokenizer = tokenizer)
aug_data = aug(train_df)

Category : 사실형-긍정-현재-확실 	 Previous size : 4743, Current size : 3794
Category : 사실형-긍정-과거-확실 	 Previous size : 7113, Current size : 5690
Category : 사실형-긍정-미래-확실 	 Previous size : 649, Current size : 649
Category : 추론형-부정-현재-확실 	 Previous size : 93, Current size : 93
Category : 예측형-긍정-미래-불확실 	 Previous size : 141, Current size : 141
Category : 추론형-긍정-현재-확실 	 Previous size : 1101, Current size : 1101
Category : 추론형-긍정-과거-확실 	 Previous size : 335, Current size : 335
Category : 추론형-긍정-현재-불확실 	 Previous size : 130, Current size : 130
Category : 대화형-긍정-미래-확실 	 Previous size : 26, Current size : 26
Category : 사실형-미정-현재-확실 	 Previous size : 18, Current size : 20
Category : 사실형-부정-과거-확실 	 Previous size : 183, Current size : 183
Category : 예측형-부정-과거-확실 	 Previous size : 1, Current size : 20
Category : 추론형-긍정-미래-확실 	 Previous size : 204, Current size : 204
Category : 사실형-긍정-미래-불확실 	 Previous size : 236, Current size : 236
Category : 대화형-긍정-현재-확실 	 Previous size : 257, Current size : 257
Category :

100%|██████████| 14675/14675 [00:01<00:00, 12730.54it/s]


In [ ]:
training_args = TrainingArguments(
    output_dir="./output_type",
    seed=seed,
    save_total_limit=2,
    save_steps = save_steps,
    num_train_epochs = 5,
    learning_rate= 1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=4,
    weight_decay=1e-4,
    logging_dir="./logs",
    logging_steps = save_steps,
    evaluation_strategy = "steps",
    eval_steps = save_steps,
    load_best_model_at_end=True,
)
test_dataset = TestDataset(data=test_df, tokenizer=tokenizer)

In [15]:
logit = 0
for i, (train_index, test_index) in enumerate(kfold_function.split(aug_data["문장"],aug_data["label"])):
    model = RobertaForSequenceClassification.from_pretrained(model_init, config=config)
    train_corpus, valid_corpus = aug_data["문장"][train_index], aug_data["문장"][test_index]
    train_label, valod_label = aug_data["label"][train_index], aug_data["label"][test_index]
    fold_train = pd.concat([train_corpus, train_label], axis =1)
    fold_valid = pd.concat([valid_corpus, valod_label], axis =1)
    train_dataset = LabelDataset(data=fold_train, tokenizer=tokenizer)
    valid_dataset = LabelDataset(data=fold_valid, tokenizer=tokenizer)

    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    )

    trainer.train()

    logit += trainer.predict(test_dataset).predictions / 5

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.bias', 'lstm.weight_hh_l0_reverse', 'lstm.weight_hh

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([    0,     8,    36,    37,    42,\n            ...\n            14642, 14645, 14655, 14659, 14661],\n           dtype='int64', length=1673). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [ ]:
with open("label_logit_focal.pickle","wb") as f:
    pickle.dump(logit, f)

In [ ]:
with open("label_logit_focal.pickle","rb") as f:
    logit = pickle.load(f)
    
result = pd.DataFrame(logit.argmax(axis=1).tolist(), columns=["type"])
test_df_result = pd.concat([test_df,result],axis=1, ignore_index=True)
test_df_result.to_csv("result.csv")